In [1]:
import torch
import pickle
import numpy as np
import networkx as nx
from PIL import ImageChops
from sentence_transformers import SentenceTransformer
from src import TypeReturn

In [2]:
m_path = 'google/embeddinggemma-300m'

In [3]:
model = SentenceTransformer(
    m_path,
    model_kwargs={
        'dtype': torch.bfloat16,
        'device_map': 'auto'
    }
)

model = torch.compile(
    model,
    mode='max-autotune',
    fullgraph=True
)

In [4]:
with open('./__output__/binaries/graph_1.pkl', 'rb') as f:
    graph: nx.Graph = pickle.load(f)

len(graph.nodes)

2044

In [5]:
node_ids = []
node_texts = []

for k, v in dict(graph.nodes.data()).items():
    node_ids.append(k)
    node_texts.append(v['text'])

node_ids= np.array(node_ids)
node_texts= np.array(node_texts)

In [6]:
embedings = model.encode(
    node_texts,
    prompt_name='document',
    batch_size=16,
    show_progress_bar=True,
    normalize_embeddings=True
)

Batches:   0%|          | 0/128 [00:00<?, ?it/s]

In [7]:
search = 'Какие философские концепции встречались в истории?'
search_emb = model.encode(
    search,
    prompt_name='query',
    normalize_embeddings=True
)

sims = embedings @ search_emb
top_k_ids = np.argsort(sims).reshape(-1)[::-1][:10]
top_k = node_ids[top_k_ids]

node_texts[top_k_ids]

array(['2.1. Философские концепции истории',
       '2.1. Философские концепции истории',
       'Глава 2. Философские концепции истории. О пониманиях истории. Цивилизационный подход. История и современность.',
       'За пределами философии сознания эти термины используются несколько иначе.',
       'Итак, бытиепредставляет собой одно из ключевых философских понятий, которое понимают либо как общее свойство, либо как качество всех существующих предметов, либо как нечто существующее само по себе, например как совокупность всего сущего (существующего) – вещей, веществ и отношений, либо как смысл конечного существования сознающего себя субъекта (человека). Таким образом, к этому центральному для метафизической философии понятию в истории есть как минимум три основных подхода – формально-логический, объективный и субъективный.',
       'Имеет место масса определений философии. Но их не следует заучивать наизусть. Философию следует понимать как познавательную деятельность в рамках наблюдае

In [8]:
[
    graph.nodes[id_]
    for id_ in top_k
]

[{'position': 146,
  'text': '2.1. Философские концепции истории',
  'node_type': <TypeReturn.LIST: 'list'>,
  'image': None,
  'level': 4},
 {'position': 160,
  'text': '2.1. Философские концепции истории',
  'node_type': <TypeReturn.HEADING: 'heading'>,
  'image': None,
  'level': 4},
 {'position': 11,
  'text': 'Глава 2. Философские концепции истории. О пониманиях истории. Цивилизационный подход. История и современность.',
  'node_type': <TypeReturn.TEXT: 'text'>,
  'image': None,
  'level': 2},
 {'position': 1736,
  'text': 'За пределами философии сознания эти термины используются несколько иначе.',
  'node_type': <TypeReturn.TEXT: 'text'>,
  'image': None,
  'level': 4},
 {'position': 611,
  'text': 'Итак, бытиепредставляет собой одно из ключевых философских понятий, которое понимают либо как общее свойство, либо как качество всех существующих предметов, либо как нечто существующее само по себе, например как совокупность всего сущего (существующего) – вещей, веществ и отношений, л

In [21]:
def get_linked_type(graph: nx.Graph, node: str, t: TypeReturn):
    res = []
    visited = set()
    for linked_node in graph[node]:
        linked_data = graph.nodes[linked_node]
        if linked_data['node_type'] == t and linked_data['text'] not in visited:
            res.append(linked_data)
            visited.add(linked_data['text'])
    return res

'''
<context>
    <images>
        Транскипции изображений
    </images>
    <parents>
        Родительские заголовки
    </parents>
    <childs>
        Зависимые Заголовки
    </childs>
    <lists>
        Списки
    </lists>
    <tables>
        Табличка
    </tables>
    <links>
    Тексты, на которые ссылаются
    </links>
    <view>
        Какие-то тексты
    </view>
</context>
<text>
Абра-Кадабра
</text>
'''

In [26]:
collaters= [
    (
        'images',
        TypeReturn.IMAGE,
        None
    ),
    (
        'parents',
        TypeReturn.HEADING,
        lambda root_node, to_node: root_node['level'] > to_node['level']
    ),
    (
        'childs',
        TypeReturn.HEADING,
        lambda root_node, to_node: root_node['level'] < to_node['level']
    ),
    (
        'lists',
        TypeReturn.LIST,
        None
    ),
    (
        'tables',
        TypeReturn.TABLE,
        None
    ),
    (
        'links',
        TypeReturn.FOOTNOTE,
        None
    ),
    (
        'view',
        TypeReturn.TEXT,
        None
    )
]


def collate_context(graph, node_ids: str):
    node = graph.nodes[node_ids]
    res = []

    for tag, type_ret, statement in collaters:
        linked_data = get_linked_type(graph, node_ids, type_ret)

        if statement is not None:
            linked_data = [
                l
                for l in linked_data
                if statement(node, l)
            ]

        linked_data = '\n'.join((
            l['text']
            for l in linked_data
        ))
        res.append(f'<{tag}>\n{linked_data}\n</{tag}>')
    return '\n'.join(res)


def get_document(graph, node_ids: str):
    return f'''
<context>
{collate_context(graph, node_ids)}
</context>
<text>
{graph.nodes[node_ids]['text']}
</text>
'''

In [27]:
print(get_document(graph, top_k[5]))


<context>
<images>
The image depicts a document with checkmarks and a gear icon. The document appears to be a checklist or a list of items, indicated by the checkmarks. The gear icon suggests that this is related to settings or configuration, possibly indicating a process for managing or adjusting settings based on the checklist.
</images>
<parents>

</parents>
<childs>

</childs>
<lists>

</lists>
<tables>

</tables>
<links>

</links>
<view>
*Философия для нефилософских специальностей. Вызовы времени и ответы философии. На чем основано и как устроено это учебное пособие.*
Существует большое количество версий понимания философии в истории, в различных странах и сообществах. История той версии философской науки, которую изучают в нашей стране, длится уже более 2,5 тысяч лет и относится к европейской культуре. Разумеется, в ней присутствуют вкрапления других культур и исторических традиций, и особенно русских, но в основе все равно лежат проблемы, сформулированные еще древними греками в